In [1]:
import tensorflow as tf

In [2]:
def master_initializer():
    uninitailized_variables=[] 
    for v in tf.global_variables():
        try :
            sess.run(v)
        except tf.errors.FailedPreconditionError:
            uninitailized_variables.append(v)
    return tf.variables_initializer(uninitailized_variables)

def print_keys(string):
    print("Collection name : {}".format(string))
    i = 0
    while True:
        try:
            print(tf.get_collection(string)[i])
            i+=1
        except IndexError:
            break;

def get_tensor_by_name(string):
    i = 0
    while True:
        try:
            if tf.global_variables()[i].name == string:
                return tf.global_variables()[i]
            i+=1
        except IndexError:
            print("No such tensor")
            return None

In [3]:
print("tf.GraphKeys.GLOBAL_VARIABLES : {}".format(tf.GraphKeys.GLOBAL_VARIABLES))
print(type(tf.GraphKeys.GLOBAL_VARIABLES))
print("tf.GraphKeys.TRAINABLE_VARIABLES : {}".format(tf.GraphKeys.TRAINABLE_VARIABLES))
print(type(tf.GraphKeys.TRAINABLE_VARIABLES))

tf.GraphKeys.GLOBAL_VARIABLES : variables
<class 'str'>
tf.GraphKeys.TRAINABLE_VARIABLES : trainable_variables
<class 'str'>


# Variables generation
    v3 : not trainable
    v4 : trainable

In [4]:
v3 = tf.Variable(1.0, name="v3", trainable=False )
v4 = tf.Variable(2.0, name="v4")

# Before restoration
-Everything are pointers(?)

In [5]:
print("tf.global_variables()")
print(tf.global_variables())
print("tf.trainable_variables()")
print(tf.trainable_variables())
print('tf.get_collection("variables")')
print(tf.get_collection("variables"))
print('tf.get_collection("trainable_variable")')
print(tf.get_collection("trainable_variables"))
print_keys("variables")
print_keys("trainable_variables")

tf.global_variables()
[<tensorflow.python.ops.variables.Variable object at 0x7f8e2fbd7a58>, <tensorflow.python.ops.variables.Variable object at 0x7f8e3c098390>]
tf.trainable_variables()
tf.get_collection("variables")
[<tensorflow.python.ops.variables.Variable object at 0x7f8e2fbd7a58>, <tensorflow.python.ops.variables.Variable object at 0x7f8e3c098390>]
tf.get_collection("trainable_variable")
Collection name : variables
Tensor("v3/read:0", shape=(), dtype=float32)
Tensor("v4/read:0", shape=(), dtype=float32)
Collection name : trainable_variables
Tensor("v4/read:0", shape=(), dtype=float32)


# Restoration

In [6]:
sess = tf.Session()
saver = tf.train.import_meta_graph('./save/model.meta')

# After restoration

In [7]:
print_keys("variables")
print_keys("trainable_variables")

Collection name : variables
Tensor("v3/read:0", shape=(), dtype=float32)
Tensor("v4/read:0", shape=(), dtype=float32)
Tensor("v1/read:0", shape=(), dtype=float32)
Tensor("v2/read:0", shape=(), dtype=float32)
Collection name : trainable_variables
Tensor("v4/read:0", shape=(), dtype=float32)
Tensor("v1/read:0", shape=(), dtype=float32)
Tensor("v2/read:0", shape=(), dtype=float32)


In [8]:
sess.run(master_initializer())
saver.restore(sess, "./save/model")
print("v3 : {}".format(sess.run(v3)))
print("v4 : {}".format(sess.run(v4)))

v1_restore = get_tensor_by_name("v1:0")
v2_restore = get_tensor_by_name("v2:0")
print("v1 : {}".format(sess.run(v1_restore)))
print("v2 : {}".format(sess.run(v2_restore)))

v3 : 1.0
v4 : 2.0
v1 : 1.3200000524520874
v2 : 1.3300000429153442


# Bonus restoration with get_variable 

In [9]:
with tf.variable_scope("hi"):
    temp = tf.get_collection("trainable_variables")[1]
    temp2 = tf.get_variable("v1_restore", shape = temp.get_shape(), initializer = tf.constant_initializer(sess.run(temp)))
    tf.get_variable_scope().reuse_variables()
    v1_restore2 = tf.get_variable(name="v1_restore", shape = None)

In [10]:
sess.run(master_initializer())
print("v1 : {}".format(sess.run(v1_restore2)))

v1 : 1.3200000524520874


# Other method to visualize variables 

In [11]:
print([v.name for v in tf.trainable_variables()])

for v in tf.global_variables():
    print(v)

['v4:0', 'v1:0', 'v2:0', 'hi/v1_restore:0']
Tensor("v3/read:0", shape=(), dtype=float32)
Tensor("v4/read:0", shape=(), dtype=float32)
Tensor("v1/read:0", shape=(), dtype=float32)
Tensor("v2/read:0", shape=(), dtype=float32)
Tensor("hi/v1_restore/read:0", shape=(), dtype=float32)
